In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
#import geo stuff
!pip install geopandas
import geopandas as gpd

In [ ]:
# load country's polygons from datahub
polygons = gpd.read_file("states.geojson")
# reduce the complexity of the shapes
polygons.geometry = polygons.geometry.simplify(.1)

In [ ]:
MA = ['Virginia', 'West Virginia', 'Pennsylvania', 'Maryland', 'Delaware', 'New Jersey']
Midwest = ['Ohio', 'Indiana', 'Michigan', 'Illinois', 'Missouri', 'Wisconsin', 'Minnesota', 'Iowa', 'Kansas', 'Nebraska', 'South Dakota', 'North Dakota']
Northeast = ['Maine', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New Hampshire', 'Vermont', 'New York', 'Pennsylvania', 'New Jersey', 'Delaware', 'Maryland']
South = ['West Virginia', 'Virginia', 'Kentucky', 'Tennessee', 'North Carolina', 'South Carolina', 'Georgia', 'Alabama','Mississippi', 'Arkansas', 'Louisiana','Florida']
Southwest = ['Texas', 'Oklahoma', 'New Mexico', 'Arizona']
West = ['Colorado', 'Wyoming', 'Montana', 'Idaho','Washington', 'Oregon', 'Utah', 'Nevada', 'California', 'Alaska', 'Hawaii']

conditions = [
    (polygons['NAME'].isin(MA)),
    (polygons['NAME'].isin(Midwest)),
    (polygons['NAME'].isin(Northeast)),
    (polygons['NAME'].isin(South)),
    (polygons['NAME'].isin(Southwest)),
    (polygons['NAME'].isin(West))
]

choices = ['Mid-Atlantic', 'Midwest', 'Northeast', 'South', 'Southwest', 'West']

polygons['Region'] = np.select(conditions, choices, default='Other')

#get rid of alaska and hawaii bc its so hard to see
polygons = polygons[polygons.NAME != 'Alaska']
polygons = polygons[polygons.NAME != 'Hawaii']

In [ ]:
city = gpd.read_file("City.geojson")
city.set_index('city')

,x,y,geometry
city,,,
Austin,30.2672,97.7431,POINT (30.267 97.743)
Houston,29.7604,95.3698,POINT (29.760 95.370)
Dallas,32.7767,96.7970,POINT (32.777 96.797)


In [ ]:
polygons.set_index('Region');

In [ ]:
filename = 'IndustryMajorLink.csv'
df_relationship = pd.read_csv(filename) 

filename = 'Major.csv'
df_major = pd.read_csv(filename)

filename = 'Region.csv'
df_region = pd.read_csv(filename)

filename = 'Industry.csv'
df_industry = pd.read_csv(filename)

In [ ]:
df_region.set_index('Region');

In [ ]:
bothregions = pd.merge(left=polygons, right=df_region, how='inner', left_on='Region', right_on='Region')

In [ ]:
bothcities = pd.merge(left=city, right=df_region, how='inner', left_on='city', right_on='Region')
bothcities

,city,x,y,geometry,Region,Year,Average,SD,Percent,Data
0,Austin,30.2672,97.7431,POINT (30.267 97.743),Austin,2020,3161,1510,48.18%,Intern
1,Austin,30.2672,97.7431,POINT (30.267 97.743),Austin,2019,3212,1425,45.47%,Intern
2,Austin,30.2672,97.7431,POINT (30.267 97.743),Austin,2018,3054,1583,44.27%,Intern
3,Austin,30.2672,97.7431,POINT (30.267 97.743),Austin,2017,2837,1012,42.86%,Intern
4,Austin,30.2672,97.7431,POINT (30.267 97.743),Austin,2020,62253,13800,37.15%,FT
5,Austin,30.2672,97.7431,POINT (30.267 97.743),Austin,2019,58218,14226,37.30%,FT
6,Austin,30.2672,97.7431,POINT (30.267 97.743),Austin,2018,59263,12912,27.55%,FT
7,Austin,30.2672,97.7431,POINT (30.267 97.743),Austin,2017,53771,14011,31.25%,FT
8,Houston,29.7604,95.3698,POINT (29.760 95.370),Houston,2020,5096,2720,20.34%,Intern
9,Houston,29.7604,95.3698,POINT (29.760 95.370),Houston,2019,4719,2760,23.12%,Intern


In [ ]:
df_industry.rename(columns={'Average': 'IAverage', 'SD': 'ISD', 'Percent': 'IPercent'}, inplace=True)
df_industry.dropna();

In [ ]:
bmerge = pd.merge(df_major,df_relationship,on='Major',how='outer')
bothmerge = pd.merge(bmerge,df_industry,on=['Industry','Year','Data'],how='outer')

In [ ]:
# 2.  we use mark_circle to have more control over visual variables
markers = alt.Chart(bothcities).mark_circle(opacity=1).encode(
    # point latitude & longitude to coordinates in geometry column
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
    #tooltip=['NAME', 'STREET', 'ZIP'],
)

# combine the two layers 
markers

alt.Chart(...)

In [ ]:
alt.data_transformers.disable_max_rows()

click = alt.selection_single(encodings=['x'])
click2 = alt.selection_single(encodings=['x'])


slider = alt.binding_range(min=2017, max=2020, step=1, name='Select Year:')
select_year = alt.selection_single(name="Year", fields=['Year'],
                                   bind=slider, init={'Year': 2020})

radio = alt.binding_radio(options=['FT', 'Intern'], name='Select Data:')
select_radio = alt.selection_single(fields=['Data'], 
                                 bind=radio, init={'Data': 'FT'})

both_t = alt.Chart(bothregions).mark_geoshape(stroke='#706545', strokeWidth=1).encode(
    tooltip=['Region',alt.Tooltip('NAME', title='State'), 'Year', 'Average', 'SD','Data'],
    color=alt.Color("Region"),    
).properties(
    width=700,
    height=500,
    title='Region Salary Statistics',
).add_selection(
    select_year,
    click,
    select_radio
).transform_filter(
    select_year
).transform_filter(
    click
).transform_filter(
    select_radio
)

both_hist = alt.Chart(bothregions).mark_bar().encode(
    y=alt.Y('Region',axis=alt.Axis(title='Region')),
    x=alt.X('Average',axis=alt.Axis(title='Salary')),
    color=alt.condition(click, 'Region', alt.value('lightgray')),
    tooltip=['Region', 'Year', 'Average', 'SD','Data'],
).properties(
    title='Distribution of Salary for Above Region Selection',
    width=400,
    height=100
).add_selection(
    select_year,
    click,
    select_radio
).transform_filter(
    select_year
).transform_filter(
    select_radio
)

both_major = alt.Chart(bothmerge).mark_bar().encode(
    x=alt.X('Major', axis=alt.Axis(title='Major')),
    y=alt.Y('Average',axis=alt.Axis(title='Average Salary')),
    tooltip=['Major','Year','Average', 'SD', 'Percent','Data'],
    color=alt.condition(click2, 'Major', alt.value('lightgrey'))
).properties(
    title='Major Salary Statistics',
    width=350
).add_selection(
    click2,
    select_year,
    select_radio
).transform_filter(
    click2
).transform_filter(
    select_year
).transform_filter(
    select_radio
)

both_industry = alt.Chart(bothmerge).mark_bar().encode(
    x=alt.X('Industry', axis=alt.Axis(title='Industry')),
    y=alt.Y('IAverage',axis=alt.Axis(title='Average Salary')),
    color=alt.condition(click2, 'Industry', alt.value('lightgrey')),
    tooltip=['Industry', 'Year', alt.Tooltip('IAverage', title='Average'),alt.Tooltip('ISD', title='SD'),alt.Tooltip('IPercent', title='Percent') ,'Data']
).properties(
    title='Industry Salary Statistics',
    width=350
).add_selection(
    select_year,
    select_radio,
    click2
).transform_filter(
    select_year
).transform_filter(
    select_radio
).transform_filter(
    click2
)

both_t + markers & both_hist & (both_major | both_industry)

alt.VConcatChart(...)

The first visualization is created through the pairing of a geomap and a horizontal bar chart. The visualization conveys relevant statistics including average annual salary and standard deviation for user selected year and employement type through the use of the slider (year) and radio button (type). The visualization can be further specified to show information regarding specific regions by making a selection by clicking on the horizontal bar chart.

The second visualization is created through the pairing of two bar charts. The visualizations convey relevant statistics including average annual salary, standard deviations and the {{percentage of people}} for the user selected year and employment type through the use of the slider (year) and radio button (type). The visualization can be further specified to show information regarding specific industries or speicific majors by a click selection on the respective bar charts.
